# 영화 추천 - 검색 모델


실제 추천 시스템은 대개 두 단계로 구성됩니다:

1. **검색 단계(Retrieval stage)**: 모든 가능한 후보군에서 초기 후보 수백 개 선택하는 역할을 합니다. 주요 목표는 사용자가 관심 없어 할 후보들을 효율적으로 걸러내는 것입니다. 검색 모델은 수백만 개의 후보와 다룰 수 있어야 하므로, 계산적으로 효율적이어야 합니다.

2. **순위 단계(Ranking stage)**: 검색 모델의 출력 결과를 세밀하게 조정하여 가능한 최선의 추천 몇 가지를 선택합니다. 이 작업의 목표는 사용자가 관심 있을 법한 아이템 세트를 좁혀서 유력한 후보 목록을 작성하는 것입니다.

이 노트북에서는 첫 번째 단계인 검색에 모델을 구현합니다.

검색 모델은 두 개의 하위 모델로 구성됩니다:

1. **쿼리 모델(Query model)**: 쿼리 feature를 사용하여 쿼리 표현(일반적으로 고정 차원의 임베딩 벡터)을 계산합니다.
2. **후보 모델(Candidate model)**: 후보 feature를 사용하여 후보 표현(동일한 크기의 벡터)을 계산합니다.

그런 다음 두 모델의 출력을 함께 곱하여 쿼리 후보 선호도 점수를 제공하며 점수가 높을수록 후보와 쿼리 간의 더 나은 일치를 나타냅니다.

이 notebook에서는 Movielens 데이터 세트를 사용하여 이러한 two-tower  모델을 구축하고 훈련합니다.

다음을 수행합니다:

1. 데이터를 가져와서 훈련 세트와 테스트 세트로 나눕니다.
2. 검색 모델을 구현합니다.
3. 모델을 피팅하고 평가합니다.
4. 효율적인 서빙을 위해 근사 최근접 이웃(ANN) 인덱스를 구축하여 모델을 내보냅니다.

이러한 과정을 통해 사용자가 관심 있을 가능성이 높은 아이템을 효율적으로 검색하는 모델을 구축하고 평가하는 방법을 배울 수 있습니다.

## dataset 준비

-  [Tensorflow Datasets](https://www.tensorflow.org/datasets)의 MovieLens 데이터셋을 사용합니다.  

- movielens/100k_ratings를 로드하면 등급 데이터가 포함된 tf.data.Dataset 객체가 생성되고 movielens/100k_movies를 로드하면 영화 데이터만 포함하는 tf.data.Dataset 객체가 생성됩니다.

- MovieLens 데이터 세트에는 미리 정의된 분할이 없기 때문에 모든 데이터는  `train` split 에 있습니다.

ratings 데이터세트에서 'user_id' 및 'movie_title' 필드만 사용합니다.  
movies 데이터세트에서는 'movie_title' 필드만 사용합니다.

## Train / Test split  

그러나 여기서는 random 분할을 사용하여 ratings의 80%를 train 세트에 넣고 20%를 테스트 세트에 넣습니다.

또한 데이터에 존재하는 고유한 사용자 ID와 영화 제목을 알아봅시다.

이는 범주형 특성의 원시 값을 모델의 임베딩 벡터에 매핑할 수 있어야 하기 때문에 중요합니다. 이를 위해서는 원시 특성 값을 연속 범위의 정수로 매핑하는 어휘가 필요합니다. 이를 통해 임베딩 테이블에서 해당 임베딩을 조회할 수 있습니다.

## 모델 구현

two-tower(Query model + Candidate model) 검색 모델을 구축합니다. 각 타워를 개별적으로 구축한 다음 최종 모델에서 결합합니다.

- 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보 item을 효율적으로 제거하는 것입니다.   
- 가능한 모든 후보 중 수백개의 초기 후보 집합을 선택하여 다음 단계의 순위 네트워크로 넘깁니다.  
- 검색 모델은 수백만개의 후보 아이템을 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*9BVJU6t9XL_GLQU8D_4wGg.png" width=400 />

### The query tower

쿼리 및 후보 표현(candidate representation)의 차원을 결정합니다. 더 큰 값을 주면 모델이 더 정확할 수 있지만 학습이 느리고 과적합되는 경향이 더 큽니다.

쿼리 모델 자체를 정의 합니다.   Keras 전처리 레이어를 사용하여 먼저 사용자 ID를 정수로 변환한 다음`Embedding` 레이어를 통해 사용자 임베딩으로 변환합니다. 이전에 구한 사용자 ID 목록을 vocabulary로 사용합니다.  

- StringLookup : string feature를 integer index로 mapping 하는 전처리 레이어

이와 같은 간단한 모델은 고전적인 행렬 분해 접근 방식과 정확히 일치합니다.  마지막에 embedding_dimension 전체 출력을 반환하는 한 표준 Keras 구성 요소를 사용하여 임의로 복잡한 모델로 쉽게 확장할 수 있습니다.

### The candidate tower

동일한 작업을 candidate tower 에 수행합니다.

### Metrics

훈련 데이터에는  positive (user, movie) 쌍이 있습니다. 우리 모델이 얼마나 좋은지 평가하려면, 모델이 계산하는 이 쌍의 선호도 점수를 다른 모든 가능한 후보들의 점수와 비교해야 합니다. 만약 positive 쌍에 대한 점수가 다른 후보들보다 높다면, 우리 모델은 매우 정확하다고 볼 수 있습니다.

이를 위해 `tfrs.metrics.FactorizedTopK` 메트릭을 사용할 수 있습니다. 이 메트릭에는 필수 인자가 하나 있는데, 바로 평가를 위한 암시적 부정 후보들로 사용되는 데이터셋입니다.

우리의 경우, 이것은 movies 데이터셋이며, 우리의 영화 모델을 통해 임베딩으로 변환됩니다.

### Loss

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 task가 있습니다.

이 경우 **손실 함수와 메트릭 계산**을 함께 묶는 편리한 래퍼인 'Retrieval' 작업 개체를 사용합니다.


task 자체는 쿼리 및 후보 임베딩을 인수로 사용하고 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 모델 구축을 간소화하는 기본 모델 클래스(`tfrs.models.Model`)를 노출합니다. 우리가 해야 할 일은 `__init__` 메소드에서 구성 요소를 설정하고 raw feature를 입력으로 받아 손실 값을 반환하는 `compute_loss` 메소드를 구현하는 것입니다.

그러면 기본 모델이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

## Fitting and evaluating

모델을 정의한 후 표준 Keras 피팅 및 평가 루틴을 사용하여 모델을 피팅하고 평가할 수 있습니다.

먼저 모델을 인스턴스화하겠습니다.

그런 다음 훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

그런 다음 모델을 훈련합니다.

모델이 학습됨에 따라 손실이 감소하고 top-k 검색 메트릭 세트가 업데이트됩니다. 이는 전체 후보 집합에서 검색된 상위 k 항목에 true positive가 있는지 여부를 알려줍니다. 예를 들어 top-5 categorical accuracy정확도 메트릭이 0.2이면 평균적으로  true positive가 검색된 상위 5개 항목의 20%에 해당한다는 것을 알 수 있습니다.

이 예에서는 평가뿐만 아니라 훈련 중에 메트릭을 평가합니다. 대규모 후보 집합에서는 이 작업이 매우 느릴 수 있으므로 훈련에서는 메트릭 계산을 끄고 평가에서만 실행하는 것이 현명할 수 있습니다.

테스트 세트의 성능이 훈련 세트의 성능보다 훨씬 나쁜 이유는 주로 두 가지 요인 때문입니다:

1. **과적합(Overfitting)**: 모델이 훈련 데이터를 외우는 경향이 있기 때문에, 본 데이터에 대해 성능이 더 좋을 가능성이 높습니다. 이러한 과적합 현상은 모델이 많은 파라미터를 가질 때 특히 강하게 나타납니다. 모델 정규화 및 사용자 및 영화 특성의 사용을 통해 모델이 본 적 없는 데이터에 대해 더 잘 일반화할 수 있도록 하여 이를 완화할 수 있습니다.

2. **이미 시청한 영화의 재추천**: 모델이 사용자가 이미 시청한 영화를 다시 추천하는 경우, 이러한 알려진 긍정적인 시청이 테스트 영화를 상위 K개 추천에서 밀어낼 수 있습니다. 이 두 번째 현상은 테스트 추천에서 이전에 본 영화를 제외함으로써 해결할 수 있습니다. 이 접근법은 추천 시스템 문헌에서 비교적 일반적이지만, 이 노트북에서는 고려하지 않습니다. 과거의 시청 이력과 맥락 정보로부터 모델이 자동으로 이러한 행동을 배울 것으로 예상하므로, 과거에 시청한 영화를 추천하지 않는 것이 중요하다면, 적절하게 지정된 모델이 이를 자동으로 학습할 것으로 기대해야 합니다. 추가적으로, 동일한 항목을 여러 번 추천하는 것(예: 언제나 인기 있는 TV 시리즈나 정기적으로 구매하는 상품)이 종종 적절할 수도 있습니다.

요약하자면, 모델이 훈련 데이터에 과적합되어 테스트 데이터에 대한 성능이 떨어지는 현상과, 이미 시청한 영화를 다시 추천하는 것을 피하기 위한 전략에 대해 설명하고 있습니다. 모델이 과거 사용자의 행동과 맥락 정보를 바탕으로 스스로 학습하여 이러한 문제를 해결할 수 있도록 하는 것이 중요하며, 때로는 동일한 아이템을 여러 번 추천하는 것이 사용자에게 유용할 수 있음을 강조하고 있습니다.

## 예측하기
이제 모델이 있으므로 예측을 할 수 있습니다. 이를 위해 tfrs.layers.factorized_top_k.BruteForce 레이어를 사용할 수 있습니다.

index.index_from_dataset 함수는 전체 영화 데이터 세트를 통해 영화 임베딩을 생성하고 이를 인덱스에 추가하여, 사용자 쿼리에 대한 영화 추천을 가능하게 하는 역할을 합니다. 이 함수는 주로 두 개의 파라미터를 받습니다.  하나는 후보 영화의 배치(batch), 다른 하나는 해당 영화를 임베딩으로 변환하는 모델입니다, 이 과정을 통해, 모델이 사용자 모델을 통해 생성된 쿼리 임베딩과 영화 모델을 통해 생성된 영화 임베딩 사이의 관계를 학습하고, 사용자에게 적합한 영화를 빠르게 검색하여 추천할 수 있습니다​​.

물론 'BruteForce' 레이어는 모델이 많은 가능한 후보를 제공하기에는 너무 느릴 것입니다. 다음 섹션에서는 대략적인 검색 인덱스를 사용하여 속도를 높이는 방법을 보여줍니다.

## Model serving

two-tower 검색 모델에서 serving에는 두 가지 구성 요소가 있습니다.

-  serving query mode. 쿼리의 feature를 가져와 쿼리 임베딩으로 변환

- serving candidate model. 쿼리 모델에 의해 생성된 쿼리에 대한 응답으로 후보를 빠르게 대략적으로 조회할 수 있는 approximate nearest neighbours(ANN) 인덱스의 형태를 취합니다.

예측 속도를 높이기 위해 대략적인 검색 인덱스(approximate retrieval index)를 내보낼 수도 있습니다. 이를 통해 수천만 명의 후보자로부터 권장 사항을 효율적으로 표시할 수 있습니다.

그렇게 하기 위해 `scann` 패키지를 사용할 수 있습니다. 이것은 TFRS의 선택적 종속성이며 `!pip install -q scann`을 호출하여 별도로 설치했습니다.

이 레이어는 _approximate_ lookup을 수행합니다. 이렇게 하면 검색 정확도가 약간 떨어지지만 큰 후보 세트에서는 훨씬 더 빨라집니다.